In [ ]:
! pip install neuralprophet

import pandas as pd
import tensorflow as tf
from neuralprophet import NeuralProphet, set_log_level
import plotly.express as px
import numpy as np
import random
import torch
import os
set_log_level("ERROR")

#시드고정
class config:
    seed = 42
    device = "cuda:0"    
        
    lr = 1e-3
    epochs = 25
    batch_size = 32
    num_workers = 4
    train_5_folds = True

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(config.seed)


def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)


seed_everything(config.seed)

tf.random.set_seed(1234)
print(tf.random.uniform([1]))
print(tf.random.uniform([1]))

print(tf.random.uniform([1], seed=1234))
print(tf.random.uniform([1], seed=1234))

os.getcwd()

#남양주시 데이터 입력
data = pd.read_csv('/content/sample_data/남양주시.csv').fillna(method='ffill').fillna(0)

#날짜변수와 종속변수 이름 변경
data = data.rename(columns={"날짜":"ds","평당가":"y"})

#각 시에 해당하는 X변수 입력
x_col_lst = ['사설학원수(개)','신도시', '미분양수', 'epu','입주물량']

#종속변수 정의
y_col_lst = ['y']

#독립변수, 종속변수 분할
X=data[x_col_lst]
Y=data[y_col_lst]

#변수들의 lag 데이터 생성
X_diff=X.diff()
X_diff.columns=X.columns+'_diff'

X_diff2=X.diff(periods=2)
X_diff2.columns=X.columns+'_diff2'

Y_diff=Y.diff()
Y_diff.columns=Y.columns+'_diff'

Y_diff2=Y.diff(periods=2)
Y_diff2.columns=Y.columns+'_diff2'

#각 독랍변수 lag데이터 결합
X=pd.concat([X,X_diff,X_diff2],axis=1)

#각 종속변수 lag데이터 결합
Y=pd.concat([Y,Y_diff,Y_diff2],axis=1)

#데이터셋 생성
data_pret=pd.concat([data['ds'][2:],X,Y],axis=1)
data_pret=data_pret.reset_index(drop=True)

#train과 test 분할
cutoff = "2020-01-01" #날짜(ds)를 통해 기준 선정
train = data_pret[data_pret['ds']<cutoff]
test = data_pret[data_pret['ds']>=cutoff]

#독립변수 이름 저장
col_lst=data_pret.columns
col_lst=col_lst.drop(['ds','y'])
col_lst=list(col_lst)
col_lst


#NeuralProphet 모델 설계
m = NeuralProphet(

growth='off', #추세 설정

yearly_seasonality=True, #년간 계절성 설정

weekly_seasonality=False, #주간 계절성 설정

daily_seasonality=False, #일간 계절성 설정

batch_size=128, #1회 학습 범위(배치사이즈) 선정

epochs=200, #학습 횟수 설정

learning_rate=0.1, #학습률 설정

n_lags= 2, #lag는 2로 설정

num_hidden_layers=4, #히든 레이어 수 설정

d_hidden=10 #은닉층 차원 수 설정

)

#정규화
m = m.add_lagged_regressor(names=col_lst, normalize="minmax")

#학습
metrics = m.fit(train, freq='MS' ,validation_df=test, progress='plot')

#MAE확인
print("SmoothL1Loss: ", metrics.SmoothL1Loss.tail(1).item())
print("MAE(Train): ", metrics.MAE.tail(1).item())
print("MAE(Test): ", metrics.MAE_val.tail(1).item())

#학습횟수에 대한 MAE값 시각화
px.line(metrics, y=['MAE', 'MAE_val'], width=800, height=400)

#test데이터 유형 변경
test.reset_index(inplace=True)

test=test.iloc[:,1:]

#LSTM을 이용해서 예측한 독립변수 가져오기
prediction = pd.read_csv('/content/sample_data/남양주시_1.csv')

#prediction 데이터 유형 변경
prediction.reset_index(inplace=True)

prediction2=prediction.iloc[:,1:]

#prediction2의 lag값을 위한 설정(위와 같음)
aa=t[x_col_lst]
bb=t[y_col_lst]

aa_diff=aa.diff()
aa_diff.columns=aa.columns+'_diff'

aa_diff2=aa.diff(periods=2)
aa_diff2.columns=aa.columns+'_diff2'

bb_diff=bb.diff()
bb_diff.columns=bb.columns+'_diff'

bb_diff2=bb.diff(periods=2)
bb_diff2.columns=bb.columns+'_diff2'

X=pd.concat([aa,aa_diff,aa_diff2],axis=1)

Y=pd.concat([bb,bb_diff,bb_diff2],axis=1)

#데이터셋
prediction3=pd.concat([prediction2['ds'][2:],X,Y],axis=1).iloc[2:]
prediction_test=prediction3.reset_index(drop=True)

#prediction를 모델에 입력
forecast = m.predict(kk)

#결과 시각화
fig = m.plot(forecast[['ds', 'y', 'yhat1']])

#결과 저장
forecast.to_excel('남양주시_forecast2.xlsx',index=False)

#오차 출력
print(abs((forecast.iloc[2][1]-forecast.iloc[2][2])/forecast.iloc[2][2])+abs((forecast.iloc[3][1]-forecast.iloc[3][2])/forecast.iloc[3][2]))
